In [2]:
# Import modules
import ee
import geemap
import geopandas as gpd
import numpy as np

In [3]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [4]:
# Open source table as a dataframe
# country_gpkg = r"data\zambia_boundary.gpkg"
country_gpkg = r"C:\Users\jedid\Documents\Code\github\crop_soil_mcda\data\zambia_boundary.gpkg"
country_gdf = gpd.read_file(country_gpkg)

# Preview the data
# country_gdf.head()

In [20]:
def get_vis_params(image, aoi):
    # reduceRegion to get the minimum ph in the AOI
    min_ph = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi,
        scale=3000,  # iSDA dataset native resolution
        maxPixels=9e8
    )

    # reduceRegion to get the maximum ph in the AOI
    max_ph = image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=3000,  # iSDA dataset native resolution
    maxPixels=9e8
    )

    # Get the result and print the minimum value
    try:
        ph_min_max = [min_ph.getInfo()['mean_0_20'], 
                            max_ph.getInfo()['mean_0_20']]
        vis_params = {'palette': ['darkred', 'red', 
                          'white', 'lightblue', 'blue', 'darkblue'],
               'min': ph_min_max[0], 'max': ph_min_max[1]}
        
    except KeyError:
        ph_min_max = [min_ph.getInfo()['constant'], 
                            max_ph.getInfo()['constant']]
        vis_params = {'palette': ['darkred', 'yellow', 'green', 'darkgreen'],
                'min': ph_min_max[0], 'max': ph_min_max[1]}
    
    return vis_params

In [14]:
# Get ranked ph suitability
def ranked_ph_suitability_image(crop, image, aoi):
    suitability_ranges = {
        'Soybean':    [(5.5, 7, 3), (5, 5.5, 2), (7, 7.5, 2), (4.5, 5, 1), (7.5, 8, 1)],
        'Groundnut':  [(5, 6.5, 3), (4.5, 5, 2), (6.5, 7, 2), (4, 4.5, 1), (7, 7.5, 1)],
        'Sugarbean':  [(5.5, 7.5, 3), (5, 5.5, 2), (7.5, 8, 2), (4.5, 5, 1), (8, 8.5, 1)],
        'Navybean':   [(5.5, 7.5, 3), (5, 5.5, 2), (7.5, 8, 2), (4.5, 5, 1), (8, 8.5, 1)],
        'Maize':      [(5.5, 7.5, 3), (5, 5.5, 2), (7.5, 8, 2), (4.5, 5, 1), (8, 8.5, 1)],
        'Cowpea':     [(5, 6.5, 3), (4.5, 5, 2), (6.5, 7, 2), (4, 4.5, 1), (7, 7.5, 1)],
        'Sunflower':  [(5, 6.5, 3), (4.5, 5, 2), (6.5, 7, 2), (4, 4.5, 1), (7, 7.5, 1)],
    }

    crop = crop.strip().replace(" ", "").title()
    ranges = suitability_ranges[crop]

    # Default suitability of 0 (no suitability)
    suitability_image = ee.Image(0).clip(aoi)

    # Apply suitability levels
    for (min_val, max_val, score) in ranges:
        mask = image.gte(min_val).And(image.lt(max_val))
        suitability_image = suitability_image.where(mask, score)

    return suitability_image

In [22]:
# Convert county from gdf to ee
country_ee = geemap.gdf_to_ee(country_gdf)

# Import ph dataset from GEE
raw_pH_dataset = ee.Image("ISDASOIL/Africa/v1/ph") \
            .select('mean_0_20').clip(country_ee)

crop = "Groundnut"

pH_dataset = raw_pH_dataset.divide(10)

# Get ranked crop suitable average ph
ranked_suitable_ph = ranked_ph_suitability_image(crop, pH_dataset, country_ee)


In [23]:
# ph visualisastion parameters
ph_vis_params = get_vis_params(pH_dataset, country_ee)
ranked_ph_vis_params = get_vis_params(ranked_suitable_ph, country_ee)

In [25]:
# Visualise layers
m = geemap.Map()
m.zoom_to_gdf(country_gdf)

m.add_layer(country_ee, name= 'Zambia')

m.add_ee_layer(pH_dataset, vis_params= ph_vis_params, name= "pH")
m.add_colorbar(ph_vis_params, orientation='vertical',  
               label= "pH", layer_name= f"pH")

m.add_ee_layer(ranked_suitable_ph, vis_params= ranked_ph_vis_params, name= f"{crop} pH Suitability")
# m.add_colorbar(ranked_ph_vis_params, orientation='vertical',  
            #    label= "pH Suitability", layer_name= f"{crop} pH Suitability")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…